# Invoking a function or do-it-yourself Agents with Bedrock

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio. You can also run on a local setup, as long as you have the right IAM credentials to invoke the Claude model via Bedrock*

---

In this demo notebook, we demonstrate an implementation of Function Calling with Anthropic's Claude models via Bedrock. This notebook is inspired by the [original work](https://drive.google.com/drive/folders/1-94Fa3HxEMkxkwKppe8lp_9-IXXvsvv1) by the Anthropic Team and modified it for use with Amazon Bedrock.

---

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers. These use natural language processing (NLP) and machine learning algorithms to understand and respond to user queries and can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. usuallythey are augmented by fetching information from various channels such as websites, social media platforms, and messaging apps which involve a complex workflow as shown below


### Chatbot using Amazon Bedrock

![Amazon Bedrock - Agents Interface](./images/agents.jpg)


### Use Cases

1. **QA** - Respond to queries based on look ups
2. **Contextual-aware chatbot** - Dialog turn by turn with extrenal look ups

### Framework for building functions/Agents with Amazon Bedrock
In Conversational interfaces such as chatbots, it is highly important to remember previous interactions, both at a short term but also at a long term level. Further we need to enhance with external tools which are needed to complete the queries. There are many providers which help to create functions or agents. We will look at a vanila implementation for how to do it yourself

### Building  - Key Elements

The first process in a building a contextual-aware chatbot is to identify the tools which can be called by the LLM's. 

Second process is the user request orchestration , interaction,  invoking and returing the results

### Architecture [Weather lookup]
We Search and look for the Latitude and Longitude and then invoke the weather app to get predictions

![Amazon Bedrock - Agents Interface](./images/weather.jpg)

## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

In [ ]:
!pip install xmltodict==0.13.0

### Anthropic Claude

#### Input

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAnswer:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### Output

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

### Bedrock model

Anthropic Claude

The key for this to work is to let LLm which is Claude models know about a set of `tools` that it has available i.e. functions it can call between a set of tags. This is possible because Anthropic's Claude models have been extensively trained on such tags in its training corpus.

Then present a way to call the tools in a step by step fashion till it gets the right answer. We create a set of callable functions in another file called `tools.py`

A sample `tools.py` is added to the same folder of this notebook and can be modified to suit your needs. Import it so that we have access to it in this notebook

In [ ]:
from utils import tools_agents

#### Helper function to pretty print

In [ ]:
from io import StringIO
import sys
import textwrap


def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))

###  Create a set of helper function

we will create a set of functions which we can the re use in our application
1. We will need to create a prompt template. This template helps Bedrock models understand the tools and how to invoke them.
2. Create a method to read the available tools and add it to the prompt being used to invoke Claude
3. Call function which will be respinsbile to actually invoke the function with the `right` parameters
4. Format Results for helping the Model leverage the results for summarization
5. Add to prompt. The result which come back need to be added to the the prompt and model invoked again to get the right results

In [ ]:
def create_prompt(tools_string, user_input):
    prompt_template = f"""
In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>

Human:
{user_input}


Assistant:
"""
    return prompt_template

### Add Tools

Recusrively add the available tools from the tools.py

In [ ]:
def add_tools():
    tools_string = ""
    for tool_spec in tools_agents.list_of_tools_specs:
        tools_string += tool_spec
    return tools_string

In [ ]:
from typing import Any
from defusedxml import ElementTree
from collections import defaultdict
print(add_tools())
# Uncomment print to test if tools is being imported correctly and your functions are correctly being interpreted via the tags.

This `call_function` will be used later to extract the name of the tool from your `tools.py` file and call it from the output of Bedrock model. A few more helper functions are defined and can be used as is without modification for your use case.

In [ ]:
def call_function(tool_name, parameters):
    func = getattr(tools_agents, tool_name)
    output = func(**parameters)
    return output

To help Bedrock models understand the results and use that for generation we need to `format_results` for deciphering

In [ ]:
def format_result(tool_name, output):
    return f"""
<function_results>
<result>
<tool_name>{tool_name}</tool_name>
<stdout>
{output}
</stdout>
</result>
</function_results>
"""

Here is where we can glue all the pieces together. Print the final prompt data to double check if the input is as expected

In [ ]:
user_input = "Can you check the weather for me in Marysville WA?"
tools_string = add_tools()
prompt_data = create_prompt(tools_string, user_input)
print_ww(prompt_data)

This next cell is to test the response of the Bedrock models based on your constructed input. Note that we have not instrumented output to call the actual functions, but this should give you an idea on how Claude's output can be parsed and the corresponding functions can be subsequently called.

In [ ]:
import traceback
def invoke_model(prompt):
    body = json.dumps({
        "prompt": prompt, 
        "max_tokens_to_sample": 1000,
        "temperature": 0,
        "stop_sequences": ["\n\nHuman:"]
    })
    modelId = "anthropic.claude-v2"

    response = boto3_bedrock.invoke_model(
        body=body, modelId=modelId, accept="application/json", contentType="application/json"
    )
    response_body = json.loads(response.get("body").read())

    #print_ww(f"invoke_model()::response_body={response_body}::")
    #print_ww(f"invoke_model()::completion={response_body.get('completion')}::")
    #print_ww(f"invoke_model()::stop_reason={response_body.get('stop_reason')}::")
    return (response_body.get("completion"), response_body.get("stop_reason"))

#completion, stop_res = invoke_model(prompt_data)


### Invoke and Add

Here we will invoke the function as per what ever the model asks us too and append the results to the prompt for history and invoke again

This process continues till the model is able to complete the task asked of it

In [ ]:
import xmltodict

def invoke_func_and_add_to_prompt(prompt, completion):
    find_index=0
    start_index = 0
    end_index = start_index + 17
    resp =  completion
    func_invoked = False
    #print(len(resp))
    while start_index < len(resp) and find_index < len(resp) and end_index < len(resp):
        start_index = resp.find("<function_calls>", find_index)
        end_index = resp.find("</function_calls>", start_index)
        if start_index < 0 or end_index < 0 or find_index <0:
            break
        func_dict = xmltodict.parse(resp[start_index:end_index+17])
        print(start_index, end_index, find_index, func_dict) #, resp[start_index:end_index+17]
        find_index = end_index+17
        func_name = func_dict['function_calls']['invoke']['tool_name']
        func_params = func_dict['function_calls']['invoke']['parameters']
        function_result = call_function(func_name, func_params)
        function_result = format_result(func_name, function_result)
        # - have to put it back as XML for Claude to pick it up
        print_ww(f"invoke_func_and_add_to_prompt():: func:invoked::{func_name}::, params={func_params}::, result={function_result}::")
        prompt += "</function_calls>"
        prompt += function_result
        func_invoked = True
    return (prompt, func_invoked)
        
#invoke_func_and_add_to_prompt(prompt_data, completion)    

### Run loop

This function is the actual orchestrator of the function calling logic. Here's how it works:

1. We kick off a loop that first calls Bedrock model with our tool use prompt with the tool specs and the user input loaded into it.
2. We get the completion from the model and check if the stop sequence for the completion was the closing tag for a function call, ```</function_calls>```
3. If the completion does in fact contain a function call, we extract out the tool name and the tool parameters from the tags.
4. We then call the function that model has decided to invoke using our helped auxillary function.
5. We take the results of the function call, format them into an tag structure, and add them back to the prompt. This works because with subsequent calls, we are basically pre-filling the output of the model and asking it to pick up where it left off, with addition data from the previous results.
6. We repeat the loop starting at step 1 with the original prompt plus the text that has been appended.
7. This process continues until model finally outputs an answer and we break the loop.
8. To avoid a never ending loop we will run the loop just a couple of times

In [ ]:

def run_loop(prompt, curr_iteration=1, max_iterations=10):
    
    # Start function calling loop -- Ideally we will have this in a while True loop but we do not want to run forever in case of issues 
    # - hence we have constrained it 
    completion = ""
    while curr_iteration < max_iterations:
        curr_iteration += 1
        
        completion, stop_res = invoke_model(prompt)
        #print_ww(f"run_loop():: prompt model invoked:prompt={prompt}::stop_reason={stop_res}: completion={completion}")
        print_ww(completion)

        # Append the completion to the end of the prommpt
        prompt += completion
        if stop_res == 'stop_sequence':
            # If Claude made a function call
            #print(completion)
            prompt, func_invoked = invoke_func_and_add_to_prompt(prompt, completion)
            if not func_invoked:
                print(f"run_loop()::Function invocation finished::Breaking:")
                break
        else:
            # If Claude did not make a function call
            # outputted answer
            print(f"run_loop()::No more functions to be run::Breaking:")
            print(completion)
            break
        
    return completion

Let's run it all together now.

In [ ]:
user_input = "Can you check the weather for me in Marysville WA?"
tools_string = add_tools()
prompt_data = create_prompt(tools_string, user_input)
weather_data = run_loop(prompt_data, curr_iteration=1, max_iterations=10)


In [ ]:
print(f"{weather_data}")

### Check another location weather

In [ ]:
user_input = "Can you check the weather for me in Seattle WA?"
tools_string = add_tools()
prompt_data = create_prompt(tools_string, user_input)
run_loop(prompt_data, curr_iteration=1, max_iterations=10)

## Next steps

In this notebook we showed some basic examples of leveraging tools and agents when invoking Amazon Bedrock models using the AWS Python SDK. You're now ready to explore the other labs to dive deeper on different use-cases and patterns.